<a href="https://colab.research.google.com/github/abbazs333/ML_dev_competitions/blob/main/end_to_end_digit_recogniser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Using transfer learning and tensorflow to classifiy numerical digits

In this project we'll try to generate a classification mode that allows us to identify numerical digits.

we'll be using dat from Digit Recogniser Kaggle competition (https://www.kaggle.com/competitions/digit-recognizer)

The workflow in this notebook will be as follows:
* Get data ready download from Kaggle, store, import)
* Prepare the data (Preprocesing the data)
* Choose a model and fit the data to it
* Evaluate the model (make predictions and compare it with the true labels)
* Improve the model throug experimentation 
* Save and reload the model once we are happy with it

For preprocessing the data we'll be using tensor flow. we'll be using a pretrained model for our project(transfer learning). Using a pretrained model reduces our cost and time spent thus it's a better option.

##Getting our workspace ready

Before we get started, let's import all necessary tools

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import tensorflow as tf
import tensorflow_hub as hub

We also need to connect our colab notebook to a GPU

##Getting the data ready
In this section we'll basically import all th necessary data and organise them in a desired form (that's eay to work with)

We have an issue though, the data that is in my google drive in a zip file so lets extract it first

In [6]:
#Ihave commented this or else everytime i run the notebood, there will be an additional copy of the data in my drive 
#!unzip "/content/drive/MyDrive/Colab Notebooks/digit_recogniser/digit-recognizer.zip" -d "/content/drive/MyDrive/Colab Notebooks/digit_recogniser/"

In [7]:
test_csv = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/digit_recogniser/test.csv")
train_csv = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/digit_recogniser/train.csv")

Let's make sure the files open 

In [10]:
test_csv.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
train_csv.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
train_csv.columns.values[:20]

array(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4',
       'pixel5', 'pixel6', 'pixel7', 'pixel8', 'pixel9', 'pixel10',
       'pixel11', 'pixel12', 'pixel13', 'pixel14', 'pixel15', 'pixel16',
       'pixel17', 'pixel18'], dtype=object)

I expected the data to be pictures but kaggle has already converted into number so we'll just use the numbers as it makes our live alot easier 🙃

before that let's do a check to see if the numbers in the dataframe are in integer/float form

In [21]:
train_csv.dtypes[:10]

label     int64
pixel0    int64
pixel1    int64
pixel2    int64
pixel3    int64
pixel4    int64
pixel5    int64
pixel6    int64
pixel7    int64
pixel8    int64
dtype: object

In [23]:
test_csv.dtypes[:10]

pixel0    int64
pixel1    int64
pixel2    int64
pixel3    int64
pixel4    int64
pixel5    int64
pixel6    int64
pixel7    int64
pixel8    int64
pixel9    int64
dtype: object

great! they are all numbers. This means we can skip the preprocessing step and move towards the next one

##Choose a model and fit the data to it

![title](https://scikit-learn.org/stable/_static/ml_map.png)

According to the above roadmap by sklearn themselves, we could you the following models:
* Linear SVC
* KNeighbours Classifier
* Ensemble Classifier

###Linear SVC